In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent, AgentState
from langchain.messages import HumanMessage, ToolMessage
from langchain.tools import tool, ToolRuntime
from langchain_mcp_adapters.client import MultiServerMCPClient
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase
from langgraph.types import Command


In [2]:
load_dotenv()

True

# Setup Tool

In [3]:
client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()



In [4]:
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:
    """Search the web for information"""

    return tavily_client.search(query=query)

In [5]:
db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def query_playlist_db(query: str) -> str:
    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

# Create state

In [6]:
class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

In [7]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")

# Create Subagents

In [8]:
# Travel agent
travel_agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="""
    You are a travel agent. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
    """
)

In [9]:
# Venue agent
venue_agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt="""
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
    """
)

In [10]:
# Playlist agent
playlist_agent = create_agent(
    model=model,
    tools=[query_playlist_db],
    system_prompt="""
    You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options.
    """
)

# Main Coordinator

In [11]:
@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travek agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["origin"]
    response = await travel_agent.ainvoke({"messages":[HumanMessage(content=f"Find the flights from {origin} to {destination}")]})

    return response["messages"][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:

    """Venue agent chooses the best venue for the given location and capacity"""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"

    response = venue_agent.invoke({"messages":[HumanMessage(content=query)]})
    return response["messages"][-1].content

@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "genre": genre, 
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )


In [12]:
coordinator = create_agent(
    model=model,
    tools = [search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt=
    """
    You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me.
    """
)

# Test

In [14]:
response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre")]
    }
)

Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring


In [15]:
from pprint import pprint
pprint(response)

{'destination': 'Paris',
 'genre': 'jazz',
 'guest_count': '100',
 'messages': [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre", additional_kwargs={}, response_metadata={}, id='36bb88b9-028f-4c26-8ba9-e416f6ce906d'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_state', 'arguments': '{"destination": "Paris", "guest_count": "100", "origin": "London", "genre": "jazz"}'}, '__gemini_function_call_thought_signatures__': {'d1f281ff-509d-4f2b-8113-8d37c01fe6fd': 'CuEBAXLI2nwVEnrwlENlQ4CKJ92pkTRNAhXFa8lN/Yq27gnZpk590RDH4qGF5Yt6DTHF8MIOxAAALHTHtWYnUV8SCuKvDG+4GPYM0H6lwWaLS0CKKKqtmDHe6e7WfwwE4V5niMI3mUVEQ5yZASTGsYnrdJyQaNAPsKhPEKQxenPhaKRjKb7jPf2zXe5ikZXonybfA7Lk1fbQoXILiPKoZAeMBCu8DrNK9CYHiu2AdilvgYuefm3qy/wEeuvrWwUEpeibDcxAboaTcT8lQmOM4AA18UzmQDJckEh6aLlQRlIu4jbe'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id

In [23]:
pprint(response["messages"][-1].content[0]["text"])

('It looks like there was an issue with the flight search. It seems to think '
 "I'm looking for a flight from London to London. Could you please confirm "
 "that you'd like flights from London to Paris, and also let me know your "
 'desired travel date?\n'
 '\n'
 'In the meantime, I have some great news!\n'
 '\n'
 'For your venue, I recommend **Les Salons Hoche** in Paris. It can '
 'comfortably accommodate 100 guests for a cocktail reception and up to 120 '
 'for a seated dinner. Reviews are positive, and while specific package prices '
 "aren't listed, catering is included, suggesting a potentially flexible and "
 'mid-range pricing structure.\n'
 '\n'
 'And for the music, I have a fantastic jazz playlist curated for you, '
 'featuring artists like Antônio Carlos Jobim, Miles Davis, and Spyro Gyra. '
 'The total duration is 6 hours, 38 minutes, and 2 seconds, with an estimated '
 'total cost of $118.80.')
